Парсер Авито для стат. исследований.

Выгружает только данные, нужные для исследований, без телефонов и прочих контактов.

Часть 1. Собиарет ссылки.

Последние правки 2022-02-06

## Библиотеки и настройки

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/' + input() + '/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки

Авито отдает только 100 страниц, 5 тыс. объявлений. Чтобы забрать все, нужно дробить запрос с помощью фильтров. 

In [ ]:
item_category = input()

In [ ]:
query = input()

In [ ]:
# разбиваю ссылку на 2 части

# первая часть ссылки, вакансии по России

url_part_1 = 'https://www.avito.ru/rossiya/?f='
#url_part_2 = 'ASgCAgECAUXGmgwXeyJmcm9tIjoxMDAwLCJ0byI6MzUwMH0&p=' # металлоискатель 
#url_part_2 = 'ASgCAgECAUXGmgwXeyJmcm9tIjozNTAwLCJ0byI6NzAwMH0&p='
#url_part_2 = 'ASgCAgECAUXGmgwYeyJmcm9tIjo3MDAwLCJ0byI6MTUwMDB9&p='
#url_part_2 = 'ASgCAgECAUXGmgwZeyJmcm9tIjoxNTAwMCwidG8iOjMyMDAwfQ&p='
url_part_2 = 'ASgCAgECAUXGmgwVeyJmcm9tIjozMjAwMCwidG8iOjB9&p='


In [ ]:
# Открываю страницу, чтобы посмотреть, сколько объявлений и страниц

driver.get(url_part_1 + url_part_2 + '1' + '&q=' + query)

In [ ]:
# количество объявлений

ads_number = (driver
              .find_element(By.TAG_NAME, 'div[data-marker="page-title"]')
              .find_element(By.TAG_NAME, 'span')
              .text.replace(' ', ''))

page_number = math.ceil(int(ads_number) / 50)

print('Количество объявлений: ' + ads_number)
print('Количество страниц: ' + str(page_number))

#### Парсер ссылок

In [ ]:
#data_urls = [] # список для данных

In [ ]:
for p in tqdm(range(1, page_number + 1)): # цикл для перехода по страницам
    
    driver.get(url_part_1 + url_part_2 + str(p) + '&q=' + query)
    
    sleep(3) # делаю паузу, даю прогрузиться
    
    
    data_blocks = driver.find_elements(By.CLASS_NAME, 'iva-item-content-rejJg')
    
    for block in data_blocks:
        
        title = block.find_element(By.CLASS_NAME, 'iva-item-titleStep-pdebR').text
        
        ad_url = block.find_element(By.TAG_NAME, 'a').get_attribute('href')
        
        price = block.find_element(By.CLASS_NAME, 'price-text-_YGDY').text
        
        try:
            description = block.find_element(By.CLASS_NAME, 'iva-item-descriptionStep-C0ty1').text
            
        except:
            description = 'none'
        
        try:
            seller = block.find_element(By.CLASS_NAME, 'style-title-_wK5H').text
            
        except:
            seller = 'none'
            
        try:
            seller_url = block.find_element(By.CLASS_NAME, 'style-link-STE_U').get_attribute('href')
            
        except:
            seller_url = 'none'
            
        try:
            seller_score = block.find_element(By.TAG_NAME, 'span[data-marker="seller-rating/score"]').text
            
        except:
            seller_score = 'none'
            
        try:
            reviews_number = block.find_element(By.TAG_NAME, 'span[data-marker="seller-rating/summary"]').text
            
        except:
            reviews_number = 'none'
                
        
        data_urls.append([item_category, title, price, description,
                          seller, seller_score, reviews_number,
                          seller_url, ad_url])

In [ ]:
# преобразовываю в датафрейм

head = ['category', 'title', 'price', 'description',
        'seller', 'seller_score', 'reviews_number',
        'seller_url', 'ad_url']

work_urls = pd.DataFrame(data_urls, columns=head)

In [ ]:
work_urls.shape

In [ ]:
work_urls.drop_duplicates(inplace=True)

In [ ]:
work_urls.shape

In [ ]:
# Записываю в файл

work_urls.to_excel(file_path + 'avito_urls_' + item_category + '.xlsx', encoding='utf-8', index=False)

In [ ]:
driver.close()